In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    # "atr",
    "trend",
    # "trend_profit",
    # "trend_multi",
    # "clusters",
    # "multi_window",
    # "validated_levels",
    # "zigzag",
    # "mean_rev",
    # "mean_rev_multi",
    # "mean_rev_vol",
    # "filter",
    # "multi_filter",
    # "filter_bidirectional",
    # "filter_one",
    # "trend_one",
    # "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2021,6,1),  test_end=datetime(2022,6,1),
    search_type='lgmm', n_trials=1000, n_models=1
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"] = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['label_method']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 both trend lgmm] modelo 0 trial 1/1000 ins=-1.000000 oos=-1.000000 avg=4.44s mem=448.36MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 2/1000 ins=-1.000000 oos=0.042078 avg=3.64s mem=463.14MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 3/1000 ins=-1.000000 oos=0.042078 avg=2.54s mem=462.36MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 4/1000 ins=-1.000000 oos=0.042078 avg=2.00s mem=463.07MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 5/1000 ins=-1.000000 oos=0.042078 avg=1.67s mem=463.24MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 6/1000 ins=-1.000000 oos=0.042078 avg=1.44s mem=463.24MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 7/1000 ins=-1.000000 oos=0.042078 avg=2.01s mem=499.89MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 8/1000 ins=-1.000000 oos=0.042078 avg=1.86s mem=473.71MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 9/1000 ins=-1.000000 oos=0.042078 avg=1.75s mem=473.67MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 10/1000 ins=-1.000000 oos=0.042078 avg=1.63s me

CatBoost is using more CPU RAM (1GiB) than the limit (1GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(65136) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(65136) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(65136) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(65136) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(65136) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(65136) > ResourceQuota(0)
CatBoost is using more CPU RAM (1GiB) than the limit (1GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(5712) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(5712) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(5712) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(5712) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.

[XAUUSD H1 both trend lgmm] modelo 0 trial 18/1000 ins=-1.000000 oos=0.042078 avg=2.62s mem=1059.84MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 19/1000 ins=-1.000000 oos=0.042078 avg=2.52s mem=1057.85MB 


CatBoost is using more CPU RAM (1.03GiB) than the limit (1GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(88160) > ResourceQuota(0)
Resource CPU 

[XAUUSD H1 both trend lgmm] modelo 0 trial 20/1000 ins=-1.000000 oos=0.042078 avg=2.59s mem=1045.97MB 


CatBoost is using more CPU RAM (1.02GiB) than the limit (1GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(83920) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(83920) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(83920) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(83920) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(83920) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(83920) > ResourceQuota(0)
CatBoost is using more CPU RAM (1.02GiB) than the limit (1GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(9468) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(9468) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(9468) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(9468) > ResourceQuota(0)
Resource CPU RAM: functionWithResource

[XAUUSD H1 both trend lgmm] modelo 0 trial 21/1000 ins=-1.000000 oos=0.042078 avg=2.64s mem=1073.46MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 22/1000 ins=-1.000000 oos=0.042078 avg=2.54s mem=1071.25MB 


CatBoost is using more CPU RAM (1.05GiB) than the limit (1GiB)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU RAM: functionWithResourceUsage.ResourceUsage(78696) > ResourceQuota(0)
Resource CPU 